# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3ef71762e0>
├── 'a' --> tensor([[ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077]])
└── 'x' --> <FastTreeValue 0x7f3ef40e32b0>
    └── 'c' --> tensor([[ 0.0073, -0.5817,  0.3366,  0.5931],
                        [ 0.5290, -0.8666, -1.1608, -2.0066],
                        [ 0.3427, -0.4823, -0.2220,  1.3879]])

In [4]:
t.a

tensor([[ 0.8046,  0.0317,  1.0132],
        [ 0.1618,  1.0767, -0.3077]])

In [5]:
%timeit t.a

75.5 ns ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3ef71762e0>
├── 'a' --> tensor([[ 0.4170,  1.1099, -0.1611],
│                   [-1.8521,  1.2830, -1.8589]])
└── 'x' --> <FastTreeValue 0x7f3ef40e32b0>
    └── 'c' --> tensor([[ 0.0073, -0.5817,  0.3366,  0.5931],
                        [ 0.5290, -0.8666, -1.1608, -2.0066],
                        [ 0.3427, -0.4823, -0.2220,  1.3879]])

In [7]:
%timeit t.a = new_value

78.7 ns ± 2.28 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077]]),
    x: Batch(
           c: tensor([[ 0.0073, -0.5817,  0.3366,  0.5931],
                      [ 0.5290, -0.8666, -1.1608, -2.0066],
                      [ 0.3427, -0.4823, -0.2220,  1.3879]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8046,  0.0317,  1.0132],
        [ 0.1618,  1.0767, -0.3077]])

In [11]:
%timeit b.a

58.2 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6857,  0.4394, -0.9618],
               [-0.4727,  2.2141,  1.5985]]),
    x: Batch(
           c: tensor([[ 0.0073, -0.5817,  0.3366,  0.5931],
                      [ 0.5290, -0.8666, -1.1608, -2.0066],
                      [ 0.3427, -0.4823, -0.2220,  1.3879]]),
       ),
)

In [13]:
%timeit b.a = new_value

572 ns ± 9.89 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

931 ns ± 24.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 174 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

263 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

244 µs ± 7.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3e4ec0d280>
├── 'a' --> tensor([[[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]],
│           
│                   [[ 0.8046,  0.0317,  1.0132],
│                    [ 0.1618,  1.0767, -0.3077]]])
└── 'x' --> <FastTreeValue 0x7f3e4ec0d0d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

41.7 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3ef40e37c0>
├── 'a' --> tensor([[ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077],
│                   [ 0.8046,  0.0317,  1.0132],
│                   [ 0.1618,  1.0767, -0.3077]])
└── 'x' --> <FastTreeValue 0x7f3e4851dee0>
    └── 'c' --> tensor([[ 0.0073, -0.5817,  0.3366,  0.5931],
                        [ 0.5290, -0.8666, -1.1608, -2.0066],
                 

In [23]:
%timeit t_cat(trees)

37.7 µs ± 986 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.3 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]],
       
               [[ 0.8046,  0.0317,  1.0132],
                [ 0.1618,  1.0767, -0.3077]]]),
    x: Batch(
           c: tensor([[[ 0.0073, -0.5817,  0.3366,  0.5931],
                       [ 0.5290, -0.8666, -1.1608, -2.0066],
                       [ 0.3427, -0.4823, -0.2220,  1.3879]],
         

In [26]:
%timeit Batch.stack(batches)

180 µs ± 6.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077],
               [ 0.8046,  0.0317,  1.0132],
               [ 0.1618,  1.0767, -0.3077]]),
    x: Batch(
           c: tensor([[ 0.0073, -0.5817,  0.3366,  0.5931],
                      [ 0.5290, -0.8666, -1.1608, -2.0066],
                      [ 0.3427, -0.4823, -0.2220,  1.3879],
                      [ 0.0073, -0.5817,  0.3366,  0.5931],
                      [ 0.5290, -

In [28]:
%timeit Batch.cat(batches)

173 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

516 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
